In [ ]:
#This is just a test

Data preprocessing

We clean the data by removing url mentions hashtags and keeping only the letters then convert to lower case

Then we tokenise the text then we remove stop words and lanel the data

In [ ]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('twitter_dataset.csv')

# Clean and preprocess the text data
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'[^A-Za-z]+', ' ', text)  # Keep only letters
    text = text.lower()  # Convert to lowercase
    return text

data['cleaned_text'] = data['text'].apply(clean_text)

# Tokenize the text
data['tokens'] = data['cleaned_text'].apply(word_tokenize)

# Remove stop words
stop_words = set(stopwords.words('english'))
data['filtered_tokens'] = data['tokens'].apply(lambda tokens: [word for word in tokens if word not in stop_words])

# Encode sentiment labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['label_encoded'] = le.fit_transform(data['sentiment'])

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(data['filtered_tokens'], data['label_encoded'], test_size=0.2, random_state=42)


then we do bigram analysos

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Create an N-gram model (e.g., bigrams)
ngram_vectorizer = CountVectorizer(ngram_range=(1, 2))
X_train_ngrams = ngram_vectorizer.fit_transform(X_train)
X_test_ngrams = ngram_vectorizer.transform(X_test)


use a multinomial nb classifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Train a classifier (e.g., Multinomial Naive Bayes)
classifier = MultinomialNB()
classifier.fit(X_train_ngrams, y_train)

# Make predictions
y_pred = classifier.predict(X_test_ngrams)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, target_names=le.classes_)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')


In [ ]:
#test